In [ ]:
import numpy as np
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

import tempfile
logdir = tempfile.mkdtemp()
print logdir

Create model weights, input placeholder x, and computation graph that we saw before


In [ ]:
b = tf.Variable(tf.zeros((100,)))
W = tf.Variable(tf.random_uniform((784, 100),
                -1, 1))

x = tf.placeholder(tf.float32, (64, 784), name="x")
h_i = tf.nn.relu(tf.matmul(x, W) + b)

Now we can get some output from tf by running our computation graph

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(h_i, {x: np.random.rand(64, 784)})

---------------------------------

## Linear regression example

In [ ]:
tf.reset_default_graph()

In [ ]:
x_batch = np.linspace(-1, 1, 101)
y_batch = x_batch * 2 + np.random.randn(*x_batch.shape) * 0.3

In [ ]:
plt.scatter(x_batch, y_batch)

Add placeholder nodes to our graph for input values $x$ and targets $y$. The only required argument to `tf.placeholder` is the datatype specification.

The second argument `shape=(None,)` indicates that these variables take on a 1-dimensional value of a *dynamic* size. We can use the `None` value in this case to allow for arbitrary batch sizes.

In [ ]:
x = tf.placeholder(tf.float32, shape=(None,), name="x")
y = tf.placeholder(tf.float32, shape=(None,), name="y")

Build a weight variable $w$ and provide an initial value. This initial value can be a tensor or some value convertible to a tensor (e.g. a numpy `ndarray`).

In [ ]:
w = tf.Variable(np.random.normal(), name="W")

Use the weight $w$ to predict a value $\hat y$ given a symbolic value $x$.

In [ ]:
y_pred = tf.mul(w, x)

We can plot the predictions of the model before training:

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    y_pred_batch = sess.run(y_pred, {x: x_batch})

In [ ]:
plt.figure(1)
plt.scatter(x_batch, y_batch)
plt.plot(x_batch, y_pred_batch)

Now let's define a cost function (MSE) and begin training.

In [ ]:
cost = tf.reduce_mean(tf.square(y_pred - y))

Register a summary op which logs observed cost values.

In [ ]:
summary_op = tf.scalar_summary("cost", cost)

Now define the optimization process.

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

Finally, prepare a summary writer instance. (Think of this as a logger.)

We provide the current graph structure as a second argument. This structure can be visualized in TensorBoard, which reads from the summary writer's output.

In [ ]:
summary_writer = tf.summary.FileWriter(logdir, sess.graph)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t in range(30):
        cost_t, summary, _ = sess.run([cost, summary_op, train_op], {x: x_batch, y: y_batch})
        summary_writer.add_summary(summary, t)
        print cost_t.mean()
        
    y_pred_batch = sess.run(y_pred, {x: x_batch})

In [ ]:
plt.figure(1)
plt.scatter(x_batch, y_batch)
plt.plot(x_batch, y_pred_batch)

[Show TensorBoard graph and events]

In [ ]:
summary_writer.flush()